## Empathy Experiments: Unified Colab Runner

This is a documented replication of the paper: Empathy Agent aimed to create a working pipline for future iterations https://arxiv.org/pdf/2503.16545

This notebook consolidates the working steps we stabilized in Colab:
- Mount Drive and sync the repo into a persistent folder
- Install compatible deps (Transformers, Accelerate, PEFT, Datasets, BitsAndBytes)
- Hugging Face login (token stored in Drive once)
- Build SFT data for Experiment 2 (EmpathyAgent replication)
- Train LoRA (4‑bit nf4) on T4 GPU with memory‑safe settings
- Generate with trained adapter and analyze results
- Persist outputs back to Drive and (optionally) zip for download

Notes:
- Keep runtime set to GPU (T4) and High RAM.
- Avoid editing tokens directly in code cells; we persist to Drive.
- We force device_map to GPU 0 to avoid CPU/disk offload errors.
- If VRAM is tight, reduce cutoff_len or increase gradient_accumulation_steps.



# Step Zero: Setting of the Runtime and imports

In [ ]:
# Minimal bootstrap: run once at session start, before the unified runner
from pathlib import Path
import os, sys, subprocess, shutil, json, gc

# Repo + Drive locations used by the unified runner
REPO_URL     = "https://github.com/limorgu/empathic-agent-cse Overlap, LCS, TF‑IDF (action-token metrics).lean.git"
DRIVE_BASE   = "/content/drive/MyDrive/colab_projects"
PROJ_DIRNAME = "empathic-agent-clean"
HF_TOKEN_FILE = f"{DRIVE_BASE}/hf_token.txt"

# Training knobs used by the runner when writing/launching the LoRA trainer
MODEL_NAME   = "meta-llama/Llama-2-7b-chat-hf"
RUN_NAME     = "llama2_lora_4bit_small"
GRAD_ACCUM   = 32
CUT_LEN      = 256
LORA_R       = 8
LORA_ALPHA   = 16

print("Bootstrap ready. Proceed to the unified runner cell.")


Bootstrap ready. Proceed to the unified runner cell.


# Step 1: Bootsrtap - Unified runner:
Drive mount → repo sync → deps → HF login → SFT → 4-bit LoRA → generate → analyze → persist


In [2]:

# 0) Runtime precheck: ensure GPU
import torch
print("CUDA available:", torch.cuda.is_available())

# 1) Mount Drive and sync repo into persistent folder
from google.colab import drive
from pathlib import Path
import os, sys, subprocess, shutil, json, gc

drive.mount("/content/drive", force_remount=False)
REPO_URL     = "https://github.com/limorgu/empathic-agent-clean.git"
DRIVE_ROOT   = "/content/drive/MyDrive/colab_projects"
PROJ_NAME    = "empathic-agent-clean"    # right folder in Drive
PROJECT_ROOT = f"{DRIVE_ROOT}/{PROJ_NAME}"

os.makedirs(DRIVE_ROOT, exist_ok=True)
if not os.path.exists(PROJECT_ROOT):
    subprocess.run(["git", "clone", REPO_URL, PROJECT_ROOT], check=True)
else:
    subprocess.run(["git", "-C", PROJECT_ROOT, "pull", "--rebase"], check=False)

# Work directory on fast local disk; mirror repo contents
!rm -rf /content/work && mkdir -p /content/work
%cd /content/work
!rsync -a "$PROJECT_ROOT/" ./

# Make imports resolve to the repo root; write outputs back into Drive path
sys.path.insert(0, PROJECT_ROOT)
os.chdir(PROJECT_ROOT)
print("PROJECT_ROOT:", PROJECT_ROOT)
!ls -la "$PROJECT_ROOT/experiments/experiment_2_empathy_agent/code" || true

# 2) Install compatible dependencies in this Colab session
# Why: avoid version conflicts and ensure bitsandbytes works on T4
%pip uninstall -y -q transformers tokenizers huggingface-hub accelerate peft datasets bitsandbytes diffusers gradio
%pip uninstall -y -q cudf-cu12 pylibcudf-cu12
%pip install -q -U \
  "huggingface-hub>=0.34,<1.0" \
  "transformers==4.46.3" \
  "tokenizers==0.20.3" \
  "accelerate>=0.33.0" \
  "peft>=0.11" \
  "datasets>=2.18" \
  "bitsandbytes>=0.43.1" \
  "safetensors>=0.4.5" \
  diffusers gradio

import transformers, huggingface_hub, accelerate, peft, datasets
print("versions:",
      "transformers", transformers.__version__,
      "hub", huggingface_hub.__version__,
      "accelerate", accelerate.__version__,
      "peft", peft.__version__,
      "datasets", datasets.__version__)
try:
    import bitsandbytes as bnb
    print("bitsandbytes:", getattr(bnb, "__version__", "?"))
except Exception as e:
    print("bitsandbytes import warning:", e)

# 3) Hugging Face login (persist token once into Drive)
from huggingface_hub import login
HF_TOKEN_FILE = f"{DRIVE_ROOT}/hf_token.txt"
if not os.path.exists(HF_TOKEN_FILE):
    try:
        tok = input("Paste your HF token (will be saved to Drive): ").strip()
    except Exception:
        tok = ""
    Path(HF_TOKEN_FILE).write_text(tok)
login(token=Path(HF_TOKEN_FILE).read_text().strip(), add_to_git_credential=True)

CUDA available: True
Mounted at /content/drive
/content/work
rsync: [sender] send_files failed to open "/content/drive/MyDrive/colab_projects/empathic-agent-clean/experiments/experiment_2_empathy_agent/output/llama2_lora_4bit_small_inference.gsheet": Operation not supported (95)
rsync error: some files/attrs were not transferred (see previous errors) (code 23) at main.c(1338) [sender=3.2.7]
PROJECT_ROOT: /content/drive/MyDrive/colab_projects/empathic-agent-clean
total 35
-rw------- 1 root root 7532 Sep 26 22:33 analyze_results.py
drwx------ 5 root root 4096 Sep 23 18:49 EmpathyAgent
-rw------- 1 root root 5750 Sep 25 18:36 finetune_lora_4bit_colab.py
-rw------- 1 root root 3717 Sep 24 19:11 finetune_lora_4bit.py
-rw------- 1 root root 3508 Sep 26 22:33 finetune_lora.py
-rw------- 1 root root 1959 Sep 26 22:33 generate_with_trained.py
-rw------- 1 root root 1586 Sep 26 22:33 prepare_sft.py
drwx------ 2 root root 4096 Sep 24 17:13 __pycache__
-rw------- 1 root root   84 Sep 24 16:49 runn

# Step 2:Environment sanity
Verify GPU and key libs; prints should all succeed.

In [ ]:
import torch, transformers, peft
print("CUDA available:", torch.cuda.is_available())
print("transformers:", transformers.__version__)
print("peft:", peft.__version__)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

CUDA available: True
transformers: 4.46.3
peft: 0.17.1


device(type='cuda')

# Step 3:  Tokenizer load
Confirms the model’s tokenizer loads and EOS exists (used for padding).

In [ ]:
from transformers import AutoTokenizer

MODEL_NAME = "add-you-token-here"  # add your Hugging Face tokens

tok = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
tok.pad_token = tok.eos_token
print("Tokenizer OK. eos_token_id:", tok.eos_token_id)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Tokenizer OK. eos_token_id: 2


# Step 4:Generate with trained Adapters - GPU and CPU loads
Load base + attach LoRA adapter (the working offload loader)
Caps GPU usage and offloads overflow to CPU if needed.
Guarantees progress with a smaller model if your T4 is near its limit.

In [ ]:
# Regenerate actions-only CSV (robust)
import os, json, csv, re, torch
from pathlib import Path
from difflib import get_close_matches
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

PROJECT_ROOT = "/content/drive/MyDrive/colab_projects/empathic-agent-clean"
BASE_MODEL  = "meta-llama/Llama-2-7b-chat-hf"
ADAPTER_DIR = f"{PROJECT_ROOT}/experiments/experiment_2_empathy_agent/output/llama2_lora_4bit"
TEST_JSON   = f"{PROJECT_ROOT}/experiments/experiment_2_empathy_agent/code/EmpathyAgent/dataset/testset_100.json"
PROMPT_L3   = f"{PROJECT_ROOT}/experiments/experiment_2_empathy_agent/code/EmpathyAgent/baseline/prompt/prompt_video_l3.txt"
ACTIONS_JSON= f"{PROJECT_ROOT}/experiments/experiment_2_empathy_agent/code/EmpathyAgent/dataset/action_list.json"
OUT_CSV     = f"{PROJECT_ROOT}/experiments/experiment_2_empathy_agent/output/llama2_lora_4bit_small_inference.csv"

def load_json_smart(p):
    for enc in ("utf-8","utf-8-sig","latin-1"):
        try:
            with open(p,"r",encoding=enc) as f: return json.load(f)
        except UnicodeDecodeError: pass
    return json.loads(open(p,"rb").read().decode("utf-8",errors="ignore"))

# Load data/prompt/actions
data   = load_json_smart(TEST_JSON)
prompt = Path(PROMPT_L3).read_text(encoding="utf-8").strip()
raw_actions = load_json_smart(ACTIONS_JSON)
actions_list = raw_actions["action_list"] if isinstance(raw_actions, dict) and "action_list" in raw_actions else raw_actions
actions_list = [str(a).strip() for a in actions_list if str(a).strip()]

# Normalization helpers
def norm(s: str) -> str:
    s = s.strip().lower()
    s = re.sub(r"[\\s]+", " ", s)
    s = re.sub(r"[\\.;:,!?]+$", "", s)
    return s

actions_norm = {norm(a): a for a in actions_list}   # normalized -> canonical
actions_norm_keys = set(actions_norm.keys())

# Load base + adapter (reuse your working offload path)
bnb = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_use_double_quant=True)
tok = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True); tok.pad_token = tok.eos_token
base = AutoModelForCausalLM.from_pretrained(BASE_MODEL, quantization_config=bnb, torch_dtype=torch.float16, device_map="auto", low_cpu_mem_usage=True)
model = PeftModel.from_pretrained(base, ADAPTER_DIR); model.eval()

def build_l3_prompt(x):
    character_info = x.get("character_info","") or x.get("character","")
    dialogue = x.get("dialogue","")
    p = prompt.format(character_info=character_info, dialogue=dialogue)
    p += (
        "\n\nRespond with ONLY grounded action tokens, one per line."
        "\nChoose strictly from the allowed list (exact tokens, no prose, no numbering)."
        "\nAllowed list:\n" + "\n".join(actions_list)
    )
    return p

# Generate and write CSV
os.makedirs(Path(OUT_CSV).parent, exist_ok=True)
rows_with_action = 0
invalid_lines_total = 0

with torch.inference_mode():
    with open(OUT_CSV,"w",newline="",encoding="utf-8") as f:
        w = csv.writer(f); w.writerow(["data_idx","response"])
        for i,x in enumerate(data):
            ip = build_l3_prompt(x)
            inp = tok(ip, return_tensors="pt", truncation=True, max_length=1024).to(model.device)  # larger to keep list
            out = model.generate(**inp, max_new_tokens=80, temperature=0.0, do_sample=False)
            # Slice only generated tokens (exclude prompt)
            gen_ids = out[0][inp["input_ids"].shape[1]:]
            resp = tok.decode(gen_ids, skip_special_tokens=True).strip()

            # Normalize + map to canonical actions
            lines = [ln.strip() for ln in resp.splitlines() if ln.strip()]
            mapped = []
            for ln in lines:
                n = norm(ln)
                if n in actions_norm_keys:
                    mapped.append(actions_norm[n])
                else:
                    # fuzzy match to nearest allowed action (threshold 0.85)
                    cands = get_close_matches(n, list(actions_norm_keys), n=1, cutoff=0.85)
                    if cands:
                        mapped.append(actions_norm[cands[0]])
                    else:
                        invalid_lines_total += 1

            mapped = list(dict.fromkeys(mapped))  # de-dup, keep order
            if mapped:
                rows_with_action += 1
            w.writerow([i, "\n".join(mapped)])
            if (i+1) % 10 == 0:
                print(f"generated {i+1}/{len(data)}")

print(f"Rows: {len(data)}, rows with ≥1 valid action: {rows_with_action}, invalid lines total: {invalid_lines_total}")
print("Wrote:", OUT_CSV)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generated 10/100
generated 20/100
generated 30/100


# Step 5: Embedding

In [ ]:
# 5. Prepare for k‑bit + embed-grad hook (expect “Embedding hook triggered: True”)
from peft import prepare_model_for_kbit_training

base.config.use_cache = False
base = prepare_model_for_kbit_training(base)
base.gradient_checkpointing_enable()

_embed_grad_flag = {"seen": False}
def _make_inputs_require_grad(module, inputs, output):
    if isinstance(output, torch.Tensor):
        output.requires_grad_(True)
        _embed_grad_flag["seen"] = True

base.get_input_embeddings().register_forward_hook(_make_inputs_require_grad)

with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
    ids = tok("hello", return_tensors="pt").to(base.device)
    _ = base(**ids)

print("Embedding hook triggered:", _embed_grad_flag["seen"])

/tmp/ipython-input-3025625321.py:16: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):


Embedding hook triggered: True


# Step 6: Apply LoRA

In [ ]:
#Apply LoRA and confirm trainables (expect trainable > 0)
from peft import LoraConfig, get_peft_model, TaskType

peft_cfg = LoraConfig(
    r=8, lora_alpha=16, lora_dropout=0.05,
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    task_type=TaskType.CAUSAL_LM,
)
model = get_peft_model(base, peft_cfg)
try:
    model.enable_input_require_grads()
except Exception:
    pass
model.config.use_cache = False
model.gradient_checkpointing_enable()

trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())
print(f"Trainable params: {trainable} / {total} ({100*trainable/total:.4f}%)")

#~20M trainables out of ~3.52B (≈0.57%) is exactly in the expected 0.3–1% range for LoRA r=8 on LLaMA-2 (q/k/v/o + MLP proj layers).

Trainable params: 19988480 / 3520401408 (0.5678%)


# Step 7:Build a tiny training sample with labels
Ensures loss is defined and backprop can run. Use your SFT jsonl if present; otherwise a minimal synthetic sample.


In [ ]:
# create json file
from pathlib import Path
import json, os

# If PROJECT_ROOT isn't set, set it here (Colab default shown)
PROJECT_ROOT = os.environ.get("PROJECT_ROOT") or "/content/drive/MyDrive/colab_projects/empathic-agent-clean"

base = Path(PROJECT_ROOT) / "experiments/experiment_2_empathy_agent"
out = base / "data" / "sft_empathyagent_mini.jsonl"
out.parent.mkdir(parents=True, exist_ok=True)

# Use the four texts we just encoded; if not in scope, make small synthetic samples
def make_minis():
    return [
        {"messages":[
            {"role":"user","content":"My friend is sad; how can I help?"},
            {"role":"assistant","content":"Offer a warm drink, listen, and suggest a short walk."}
        ]},
        {"messages":[
            {"role":"user","content":"I argued with my sibling; what now?"},
            {"role":"assistant","content":"Acknowledge feelings, apologize for your part, and propose a calm chat."}
        ]},
        {"messages":[
            {"role":"user","content":"I feel overwhelmed at work."},
            {"role":"assistant","content":"Take a short break, write tasks, and ask a colleague for support."}
        ]},
        {"messages":[
            {"role":"user","content":"I worry about a big exam."},
            {"role":"assistant","content":"Create a study plan, practice breathing, and plan a small reward."}
        ]},
    ]

records = make_minis()
with out.open("w", encoding="utf-8") as w:
    for r in records:
        w.write(json.dumps(r, ensure_ascii=False) + "\n")

print("Wrote:", out)

Wrote: /content/drive/MyDrive/colab_projects/empathic-agent-clean/experiments/experiment_2_empathy_agent/data/sft_empathyagent_mini.jsonl


In [ ]:
from pathlib import Path
import json

# Prefer your dataset; fallback to synthetic
SFT_JSONL = Path(PROJECT_ROOT)/"experiments/experiment_2_empathy_agent/data/sft_empathyagent.jsonl"

def sample_text():
    return "<|user|>\nHow can I comfort my friend?\n<|assistant|>\nOffer a warm tea and a supportive sentence.\n<|end|>\n"

records = []
if SFT_JSONL.exists():
    # read just a few lines
    with SFT_JSONL.open("r", encoding="utf-8") as f:
        for i, line in enumerate(f):
            if i >= 4: break
            obj = json.loads(line)
            msg = obj.get("messages", [])
            if msg:
                parts = []
                for m in msg:
                    parts.append(f"<|{m.get('role','user')}|>\n{m.get('content','')}\n")
                records.append("".join(parts) + "<|end|>\n")
else:
    records = [sample_text()] * 4

enc_items = []
for t in records:
    enc = tok(t, truncation=True, max_length=256)
    enc_items.append({
        "input_ids": enc["input_ids"],
        "attention_mask": enc["attention_mask"],
        "labels": enc["input_ids"],   # critical for loss/backprop
    })

print("Prepared items:", len(enc_items), "example lengths:", [len(x["input_ids"]) for x in enc_items])

Prepared items: 4 example lengths: [171, 173, 196, 175]


In [ ]:

# Quick verify head+count
from itertools import islice, count
print("Exists:", out.exists(), "Size:", out.stat().st_size if out.exists() else 0)
with out.open("r", encoding="utf-8") as f:
    print("First 2 lines:")
    for line in islice(f, 2):
        print(line.rstrip())

print("Line count:", sum(1 for _ in open(out, "r", encoding="utf-8")))

Exists: True Size: 718
First 2 lines:
{"messages": [{"role": "user", "content": "My friend is sad; how can I help?"}, {"role": "assistant", "content": "Offer a warm drink, listen, and suggest a short walk."}]}
{"messages": [{"role": "user", "content": "I argued with my sibling; what now?"}, {"role": "assistant", "content": "Acknowledge feelings, apologize for your part, and propose a calm chat."}]}
Line count: 4


In [ ]:
# build the full 10k file later
from experiments.experiment_2_empathy_agent.code.prepare_sft import main as build_sft
build_sft()

Wrote /content/drive/MyDrive/colab_projects/empathic-agent-clean/experiments/experiment_2_empathy_agent/data/sft_empathyagent.jsonl records: 10000


# Step 8: Smoke-test
a single backward pass (no trainer yet) - proves the computation graph allows grads

In [ ]:
import torch
batch_ids = tok("test", return_tensors="pt").to(model.device)
batch = {
    "input_ids": batch_ids["input_ids"],
    "attention_mask": batch_ids["attention_mask"],
    "labels": batch_ids["input_ids"],
}
model.train()
out = model(**batch)
loss = out.loss
print("Loss:", float(loss))
loss.backward()
print("Backward OK.")
model.zero_grad(set_to_none=True)

/tmp/ipython-input-2430980668.py:11: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at /pytorch/torch/csrc/autograd/generated/python_variable_methods.cpp:835.)
  print("Loss:", float(loss))


Loss: 11.397171020507812
Backward OK.


# Step 9:  Tiny trainer run
verify trainer integration and that backward works under Trainer

In [ ]:
from transformers import TrainingArguments, Trainer
from torch.utils.data import Dataset

class TinyDS(Dataset):
    def __init__(self, items): self.items = items
    def __len__(self): return len(self.items)
    def __getitem__(self, idx):
        x = self.items[idx]
        return {k: torch.tensor(x[k]) for k in x.keys()}

train_ds = TinyDS(enc_items)

args = TrainingArguments(
    output_dir=str(Path(PROJECT_ROOT)/"experiments/experiment_2_empathy_agent/output/tmp_run"),
    run_name="smoke",
    learning_rate=2e-5,
    max_steps=2,                               # tiny run
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    logging_steps=1,
    bf16=torch.cuda.is_available(),
    fp16=not torch.cuda.is_available(),
    report_to=[],
    remove_unused_columns=False,               # keep labels
)

from transformers import DataCollatorForLanguageModeling
collator = DataCollatorForLanguageModeling(tokenizer=tok, mlm=False)

trainer = Trainer(model=model, args=args, train_dataset=train_ds, data_collator=collator)
trainer.train()                                # should finish without grad errors
print("Tiny Trainer run OK.")

'''
What this means:
This smoke run used a tiny dataset and max_steps=2. Small oscillations (even an uptick) are normal with so few steps and random initial batches. The key pass/fail is that it ran forward+backward without “does not require grad” errors and saved a checkpoint directory.
What to check quickly:
A small output folder under experiments/experiment_2_empathy_agent/output/tmp_run with trainer_state.json and checkpoint-* confirms the Trainer pipeline worked.
'''

max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss
1,2.713500
2,2.817000


Tiny Trainer run OK.


'\nWhat this means:\nThis smoke run used a tiny dataset and max_steps=2. Small oscillations (even an uptick) are normal with so few steps and random initial batches. The key pass/fail is that it ran forward+backward without “does not require grad” errors and saved a checkpoint directory.\nWhat to check quickly:\nA small output folder under experiments/experiment_2_empathy_agent/output/tmp_run with trainer_state.json and checkpoint-* confirms the Trainer pipeline worked.\n'

# Step 10:  Full trainer write + real train + generate/analyze
his writes the trainer file with the same “prepared” setup and runs a real (longer) training.


In [ ]:
from pathlib import Path
import os

assert "PROJECT_ROOT" in globals(), "PROJECT_ROOT must be set"
TRAINER_PATH = Path(PROJECT_ROOT) / "experiments/experiment_2_empathy_agent/code/finetune_lora_4bit_colab.py"
TRAINER_PATH.parent.mkdir(parents=True, exist_ok=True)

code = f"""
import os, json, torch
from dataclasses import dataclass
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    TrainingArguments, Trainer,
    DataCollatorForLanguageModeling, BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training

HERE = os.getcwd()
BASE = os.path.abspath(os.path.join(HERE, "experiments", "experiment_2_empathy_agent"))

@dataclass
class Config:
    model_name: str = "{MODEL_NAME}"
    data_path: str = os.path.join(BASE, "data", "sft_empathyagent.jsonl")
    output_dir: str = os.path.join(BASE, "output", "llama2_lora_4bit")
    run_name: str = "llama2_lora_4bit_small"
    lr: float = 2e-5
    epochs: int = 1
    per_device_train_batch_size: int = 1
    gradient_accumulation_steps: int = 32
    cutoff_len: int = 256
    lora_r: int = 8
    lora_alpha: int = 16
    lora_dropout: float = 0.05

def _compute_dtype():
    if torch.cuda.is_available():
        major, _ = torch.cuda.get_device_capability()
        return torch.bfloat16 if major >= 8 else torch.float16
    return torch.float16

def load_jsonl(path):
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            s = line.strip()
            if s:
                yield json.loads(s)

def _messages_to_text(messages):
    parts = []
    for m in messages:
        parts.append(f"<|{{m.get('role','user')}}|>\\n{{m.get('content','')}}\\n")
    return "".join(parts) + "<|end|>\\n"

def format_examples(records, tok, cutoff):
    ds = []
    for rec in records:
        if "messages" in rec and isinstance(rec["messages"], list):
            text = _messages_to_text(rec["messages"])
        elif "input" in rec and "output" in rec:
            user = str(rec.get("input","")); assistant = str(rec.get("output",""))
            text = f"<|user|>\\n{{user}}\\n<|assistant|>\\n{{assistant}}\\n<|end|>\\n"
        else:
            continue
        enc = tok(text, truncation=True, max_length=cutoff)
        ds.append({{"input_ids": enc["input_ids"], "attention_mask": enc["attention_mask"], "labels": enc["input_ids"]}})
    return ds

def main(cfg=Config()):
    print("[7.0] Trainer starting with model:", cfg.model_name)
    print("[7.0] Data path:", cfg.data_path)
    if not os.path.exists(cfg.data_path):
        raise FileNotFoundError(f"sft file not found: {{cfg.data_path}}")

    tok = AutoTokenizer.from_pretrained(cfg.model_name, use_fast=True)
    tok.pad_token = tok.eos_token

    bnb = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=_compute_dtype(),
    )
    base = AutoModelForCausalLM.from_pretrained(
        cfg.model_name, quantization_config=bnb,
        torch_dtype=torch.float16, device_map={{"": 0}}, low_cpu_mem_usage=True,
    )

    base.config.use_cache = False
    base = prepare_model_for_kbit_training(base)
    base.gradient_checkpointing_enable()

    _flag = {{"seen": False}}
    def _make_inputs_require_grad(module, inputs, output):
        if isinstance(output, torch.Tensor):
            output.requires_grad_(True); _flag["seen"] = True
    base.get_input_embeddings().register_forward_hook(_make_inputs_require_grad)

    peft_cfg = LoraConfig(
        r=cfg.lora_r, lora_alpha=cfg.lora_alpha, lora_dropout=cfg.lora_dropout,
        target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
        task_type=TaskType.CAUSAL_LM,
    )
    model = get_peft_model(base, peft_cfg)
    try: model.enable_input_require_grads()
    except Exception: pass
    model.config.use_cache = False
    model.gradient_checkpointing_enable()

    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total = sum(p.numel() for p in model.parameters())
    pct = 100*trainable/total if total else 0.0
    print(f"[7.0] Trainable params: {{trainable}} / {{total}} ({{pct:.4f}}%)")
    if trainable == 0:
        raise RuntimeError("No trainable params — check target_modules.")

    # trigger embed hook once
    ids = tok("hello", return_tensors="pt").to(model.device)
    _ = model(**ids)
    print("[7.0] Embedding hook seen:", _flag["seen"])

    records = list(load_jsonl(cfg.data_path))
    print("[7.0] Loaded records:", len(records))
    if not records:
        raise RuntimeError("No records loaded from sft file")

    train_ds = format_examples(records, tok, cfg.cutoff_len)
    print("[7.0] Encoded examples:", len(train_ds), "example[0] len:", len(train_ds[0]["input_ids"]) if train_ds else 0)
    collator = DataCollatorForLanguageModeling(tokenizer=tok, mlm=False)

    args = TrainingArguments(
        output_dir=cfg.output_dir, run_name=cfg.run_name, learning_rate=cfg.lr,
        num_train_epochs=cfg.epochs,
        per_device_train_batch_size=cfg.per_device_train_batch_size,
        gradient_accumulation_steps=cfg.gradient_accumulation_steps,
        logging_steps=10, save_steps=200, save_total_limit=2,
        bf16=torch.cuda.is_available(), fp16=not torch.cuda.is_available(),
        report_to=[], remove_unused_columns=False,
    )

    from torch.utils.data import Dataset
    class JsonlDS(Dataset):
        def __init__(self, items): self.items = items
        def __len__(self): return len(self.items)
        def __getitem__(self, i):
            x = self.items[i]
            return {{"input_ids": torch.tensor(x["input_ids"]),
                     "attention_mask": torch.tensor(x["attention_mask"]),
                     "labels": torch.tensor(x["labels"])}}

    ds = JsonlDS(train_ds)
    print("[7.0] Dataset ready. READY_TO_TRAIN")
    # note: training starts in later steps
    return model, tok, ds, collator, args

if __name__ == "__main__":
    main()
"""
TRAINER_PATH.write_text(code, encoding="utf-8")
print("Trainer written to:", TRAINER_PATH)

Trainer written to: /content/drive/MyDrive/colab_projects/empathic-agent-clean/experiments/experiment_2_empathy_agent/code/finetune_lora_4bit_colab.py


# Step 11:  — Import trainer and build objects (no training yet)

Imports the trainer module and runs main to construct model/dataset/args only.
Prints READY_TO_TRAIN and sanity info.
Expected:
“[7.0] Trainable params: … > 0”
“[7.0] Embedding hook seen: True”
“[7.0] Loaded records: … > 0”
“READY_TO_TRAIN”

In [ ]:
from experiments.experiment_2_empathy_agent.code.finetune_lora_4bit_colab import Config as Cfg, main as trainer_build

cfg = Cfg()  # adjust if you want different cutoff_len/grad_accum/etc.
model, tok, train_ds, collator, args = trainer_build(cfg)

[7.0] Trainer starting with model: meta-llama/Llama-2-7b-chat-hf
[7.0] Data path: /content/drive/MyDrive/colab_projects/empathic-agent-clean/experiments/experiment_2_empathy_agent/data/sft_empathyagent.jsonl


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

[7.0] Trainable params: 19988480 / 3520401408 (0.5678%)
[7.0] Embedding hook seen: True
[7.0] Loaded records: 10000
[7.0] Encoded examples: 10000 example[0] len: 171
[7.0] Dataset ready. READY_TO_TRAIN


Short warm-up training (max_steps=10)
Reuses the built objects, overrides max_steps for a quick 10-step run.
Confirms end-to-end training works before a longer run.


In [ ]:
from transformers import Trainer

warm_args = args
warm_args.max_steps = 10  # short warm-up
print("[7.2] Starting warm-up training for", warm_args.max_steps, "steps")

trainer = Trainer(model=model, args=warm_args, train_dataset=train_ds, data_collator=collator)
trainer.train()
print("[7.2] Warm-up done. Checkpoints at:", warm_args.output_dir)

max_steps is given, it will override any value given in num_train_epochs


[7.2] Starting warm-up training for 10 steps


Step,Training Loss
10,2.953900


[7.2] Warm-up done. Checkpoints at: /content/drive/MyDrive/colab_projects/empathic-agent-clean/experiments/experiment_2_empathy_agent/output/llama2_lora_4bit


# Step 11: Clears VRAM, runs the proper full run (epochs etc. from Config).

Expected:
Longer logs; multiple checkpoints; final model saved in output/llama2_lora_4bit/ with adapter_config.json and adapter_model.safetensors.

In [ ]:
import gc, torch
gc.collect();
try: torch.cuda.empty_cache()
except Exception: pass

print("[7.3] Full training starting. Epochs:", cfg.epochs)
trainer = Trainer(model=model, args=args, train_dataset=train_ds, data_collator=collator)
trainer.train()
trainer.save_model(args.output_dir); tok.save_pretrained(args.output_dir)
print("[7.3] Full training done. Saved to:", args.output_dir)

max_steps is given, it will override any value given in num_train_epochs


[7.3] Full training starting. Epochs: 1


Step,Training Loss
10,2.784400


[7.3] Full training done. Saved to: /content/drive/MyDrive/colab_projects/empathic-agent-clean/experiments/experiment_2_empathy_agent/output/llama2_lora_4bit


# Step 12 — Post-train checks
What it does:
Verifies required files exist; prints count.
Expected:
Both files found.

In [ ]:
from pathlib import Path
out_dir = Path(args.output_dir)
need = ["adapter_config.json", "adapter_model.safetensors"]
found = [p for p in need if (out_dir / p).exists()]
print("[7.4] Found:", found, "Missing:", list(set(need) - set(found)))

[7.4] Found: ['adapter_config.json', 'adapter_model.safetensors'] Missing: []


# Step 8. Generate a CSV with LoRA


Note: For OOM (out of memoory): lower cutoff_len (e.g., 256 → 192/128) or increase gradient_accumulation_steps; GPU offload is slower but works.

In [ ]:
# Reset CUDA memory and make allocator more flexible
import os, gc, torch
for v in ("model","base"):
    if v in globals(): del globals()[v]
gc.collect();
if torch.cuda.is_available(): torch.cuda.empty_cache()
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128,expandable_segments:True"

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

PROJECT_ROOT = "/content/drive/MyDrive/colab_projects/empathic-agent-clean"
BASE_MODEL  = "meta-llama/Llama-2-7b-chat-hf"
ADAPTER_DIR = f"{PROJECT_ROOT}/experiments/experiment_2_empathy_agent/output/llama2_lora_4bit"

def _dtype():
    if torch.cuda.is_available():
        major, _ = torch.cuda.get_device_capability()
        return torch.bfloat16 if major >= 8 else torch.float16
    return torch.float16

bnb = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=_dtype(),
)

tok = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
tok.pad_token = tok.eos_token

# NO "disk" in max_memory; just GPU index and "cpu"
base = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb,
    torch_dtype="auto",
    device_map="auto",
    max_memory={0: "10GiB", "cpu": "48GiB"},   # try 10GiB; raise/lower if needed
    offload_folder="/content/offload",         # enables CPU/disk offload automatically
    low_cpu_mem_usage=True,
    use_safetensors=True,
)

model = PeftModel.from_pretrained(base, ADAPTER_DIR)
model.eval()
print("Loaded with CPU offload; adapter attached.")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Loaded with CPU offload; adapter attached.


# Step 10: Regenerate CSV with only the L3 Action

In [4]:


'''# Regenerate actions-only CSV (robust)
import os, json, csv, re, torch
from pathlib import Path
from difflib import get_close_matches
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

PROJECT_ROOT = "/content/drive/MyDrive/colab_projects/empathic-agent-clean"
BASE_MODEL  = "meta-llama/Llama-2-7b-chat-hf"
ADAPTER_DIR = f"{PROJECT_ROOT}/experiments/experiment_2_empathy_agent/output/llama2_lora_4bit"
TEST_JSON   = f"{PROJECT_ROOT}/experiments/experiment_2_empathy_agent/code/EmpathyAgent/dataset/testset_100.json"
PROMPT_L3   = f"{PROJECT_ROOT}/experiments/experiment_2_empathy_agent/code/EmpathyAgent/baseline/prompt/prompt_video_l3.txt"
ACTIONS_JSON= f"{PROJECT_ROOT}/experiments/experiment_2_empathy_agent/code/EmpathyAgent/dataset/action_list.json"
OUT_CSV     = f"{PROJECT_ROOT}/experiments/experiment_2_empathy_agent/output/llama2_lora_4bit_small_inference.csv"

def load_json_smart(p):
    for enc in ("utf-8","utf-8-sig","latin-1"):
        try:
            with open(p,"r",encoding=enc) as f: return json.load(f)
        except UnicodeDecodeError: pass
    return json.loads(open(p,"rb").read().decode("utf-8",errors="ignore"))

# Load data/prompt/actions
data   = load_json_smart(TEST_JSON)
prompt = Path(PROMPT_L3).read_text(encoding="utf-8").strip()
raw_actions = load_json_smart(ACTIONS_JSON)
actions_list = raw_actions["action_list"] if isinstance(raw_actions, dict) and "action_list" in raw_actions else raw_actions
actions_list = [str(a).strip() for a in actions_list if str(a).strip()]

# Normalization helpers
def norm(s: str) -> str:
    s = s.strip().lower()
    s = re.sub(r"[\\s]+", " ", s)
    s = re.sub(r"[\\.;:,!?]+$", "", s)
    return s

actions_norm = {norm(a): a for a in actions_list}   # normalized -> canonical
actions_norm_keys = set(actions_norm.keys())

# Load base + adapter (reuse your working offload path)
bnb = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_use_double_quant=True)
tok = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True); tok.pad_token = tok.eos_token
base = AutoModelForCausalLM.from_pretrained(BASE_MODEL, quantization_config=bnb, torch_dtype=torch.float16, device_map="auto", low_cpu_mem_usage=True)
model = PeftModel.from_pretrained(base, ADAPTER_DIR); model.eval()

def build_l3_prompt(x):
    character_info = x.get("character_info","") or x.get("character","")
    dialogue = x.get("dialogue","")
    p = prompt.format(character_info=character_info, dialogue=dialogue)
    p += (
        "\n\nRespond with ONLY grounded action tokens, one per line."
        "\nChoose strictly from the allowed list (exact tokens, no prose, no numbering)."
        "\nAllowed list:\n" + "\n".join(actions_list)
    )
    return p

# Generate and write CSV
os.makedirs(Path(OUT_CSV).parent, exist_ok=True)
rows_with_action = 0
invalid_lines_total = 0

with torch.inference_mode():
    with open(OUT_CSV,"w",newline="",encoding="utf-8") as f:
        w = csv.writer(f); w.writerow(["data_idx","response"])
        for i,x in enumerate(data):
            ip = build_l3_prompt(x)
            inp = tok(ip, return_tensors="pt", truncation=True, max_length=1024).to(model.device)  # larger to keep list
            out = model.generate(**inp, max_new_tokens=80, temperature=0.0, do_sample=False)
            # Slice only generated tokens (exclude prompt)
            gen_ids = out[0][inp["input_ids"].shape[1]:]
            resp = tok.decode(gen_ids, skip_special_tokens=True).strip()

            # Normalize + map to canonical actions
            lines = [ln.strip() for ln in resp.splitlines() if ln.strip()]
            mapped = []
            for ln in lines:
                n = norm(ln)
                if n in actions_norm_keys:
                    mapped.append(actions_norm[n])
                else:
                    # fuzzy match to nearest allowed action (threshold 0.85)
                    cands = get_close_matches(n, list(actions_norm_keys), n=1, cutoff=0.85)
                    if cands:
                        mapped.append(actions_norm[cands[0]])
                    else:
                        invalid_lines_total += 1

            mapped = list(dict.fromkeys(mapped))  # de-dup, keep order
            if mapped:
                rows_with_action += 1
            w.writerow([i, "\n".join(mapped)])
            if (i+1) % 10 == 0:
                print(f"generated {i+1}/{len(data)}")

print(f"Rows: {len(data)}, rows with ≥1 valid action: {rows_with_action}, invalid lines total: {invalid_lines_total}")
print("Wrote:", OUT_CSV)
'''

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


generated 10/100
generated 20/100
generated 30/100
generated 40/100
generated 50/100
generated 60/100


KeyboardInterrupt: 

# Rescore L3 actions CSV (write …reference_based_score.csv)

In [ ]:
# Score after verifying action-only format against baseline
from experiments.experiment_2_empathy_agent.code.EmpathyAgent.baseline.overlap import Overlap, TF_IDF, LCS
import json, csv

TEST_JSON = f"{PROJECT_ROOT}/experiments/experiment_2_empathy_agent/code/EmpathyAgent/dataset/testset_100.json"

def load_json_smart(p):
    for enc in ("utf-8","utf-8-sig","latin-1"):
        try:
            with open(p,"r",encoding=enc) as f: return json.load(f)
        except UnicodeDecodeError: pass
    return json.loads(open(p,"rb").read().decode("utf-8",errors="ignore"))

data = load_json_smart(TEST_JSON)
resp = {}
with open(CSV_PATH, encoding="utf-8") as f:
    for r in csv.DictReader(f):
        i = (r.get("data_idx") or "").strip()
        t = (r.get("response") or "").strip()
        if i.isdigit() and t: resp[i] = t

ov,lcs,tf = Overlap(), LCS(), TF_IDF()
print("Overlap:", ov.score(resp, TEST_JSON))
print("LCS:",     lcs.score(resp, TEST_JSON))
print("TF-IDF:",  tf.score(resp, TEST_JSON))

100%|██████████| 100/100 [00:00<00:00, 112387.57it/s]


Average Overlapping Accuracy: 0.0
Overlap: 0.0


100%|██████████| 100/100 [00:00<00:00, 168310.75it/s]


Average LCS Similarity: 0.0
LCS: 0.0


100%|██████████| 100/100 [00:00<00:00, 163904.03it/s]

Average TF_IDF Similarity: 0.0
TF-IDF: 0.0


In [ ]:
# L3 rescore (Overlap/LCS/TF-IDF) for your LoRA CSV
import os, sys, csv, json
from pathlib import Path

PROJECT_ROOT = "/content/drive/MyDrive/colab_projects/empathic-agent-clean"
BASELINE_DIR = f"{PROJECT_ROOT}/experiments/experiment_2_empathy_agent/code/EmpathyAgent/baseline"
TEST_JSON    = f"{PROJECT_ROOT}/experiments/experiment_2_empathy_agent/code/EmpathyAgent/dataset/testset_100.json"
IN_CSV       = f"{PROJECT_ROOT}/experiments/experiment_2_empathy_agent/output/llama2_lora_4bit_small_inference.csv"
OUT_SCORE    = f"{PROJECT_ROOT}/experiments/experiment_2_empathy_agent/output/llama2_lora_4bit_small_reference_based_score.csv"

sys.path.insert(0, BASELINE_DIR)
from overlap import Overlap, TF_IDF, LCS  # baseline metric implementations

# load generations
resp = {}
with open(IN_CSV, encoding="utf-8") as f:
    for r in csv.DictReader(f):
        i = (r.get("data_idx") or "").strip()
        t = (r.get("response") or "").strip()
        if i.isdigit():
            resp[i] = t

ov, lcs, tf = Overlap(), LCS(), TF_IDF()
scores = {
    "Overlap": ov.score(resp, TEST_JSON),
    "LCS":     lcs.score(resp, TEST_JSON),
    "TF-IDF":  tf.score(resp, TEST_JSON),
}
print("L3 reference-based:", scores)

# write CSV
with open(OUT_SCORE, "w", newline="", encoding="utf-8") as f:
    w = csv.writer(f); w.writerow(["Metric","Score"])
    for k, v in scores.items(): w.writerow([k, float(v)])
print("Wrote:", OUT_SCORE)

Rows: 60, rows with ≥1 valid action: 0, invalid lines total: 0
Tip: rows with zero valid actions will contribute 0 to all metrics.


# Step 12: Score L1 with BERTScore (reference-based)

In [ ]:
# L1 BERTScore (reference-based)
import os, csv, json
from pathlib import Path
from bert_score import score

PROJECT_ROOT = "/content/drive/MyDrive/colab_projects/empathic-agent-clean"
TEST_JSON    = f"{PROJECT_ROOT}/experiments/experiment_2_empathy_agent/code/EmpathyAgent/dataset/testset_100.json"
IN_CSV       = f"{PROJECT_ROOT}/experiments/experiment_2_empathy_agent/output/llama2_lora_4bit_small_L1_inference.csv"
OUT_SCORE    = f"{PROJECT_ROOT}/experiments/experiment_2_empathy_agent/output/llama2_lora_4bit_small_L1_reference_based_score.csv"

def load_json(p):
    for enc in ("utf-8","utf-8-sig","latin-1"):
        try:
            with open(p,"r",encoding=enc) as f: return json.load(f)
        except UnicodeDecodeError: pass
    return json.loads(open(p,"rb").read().decode("utf-8",errors="ignore"))

data = load_json(TEST_JSON)
refs = [ (x.get("scenario_understanding") or x.get("reference") or "").strip() for x in data ]  # adapt to dataset field
gens = [""] * len(data)

with open(IN_CSV, encoding="utf-8") as f:
    for r in csv.DictReader(f):
        i = (r.get("data_idx") or "").strip()
        t = (r.get("response") or "").strip()
        if i.isdigit():
            gens[int(i)] = t

# keep aligned pairs only
pairs = [(g, r) for g, r in zip(gens, refs)]
gens2, refs2 = zip(*pairs)

P, R, F1 = score(list(gens2), list(refs2), lang="en", model_type="roberta-large", verbose=False)
val = float(F1.mean().item())
print("BERTScore (F1):", val)

with open(OUT_SCORE, "w", newline="", encoding="utf-8") as f:
    w = csv.writer(f); w.writerow(["Metric","Score"]); w.writerow(["BERTScore", val])
print("Wrote:", OUT_SCORE)

In [ ]:
import os, glob
print(os.path.abspath(OUT_CSV))
print(glob.glob(str(Path(OUT_CSV).parent / "*.csv")))

/content/drive/MyDrive/colab_projects/empathic-agent-clean/experiments/experiment_2_empathy_agent/output/llama2_lora_4bit_small_inference.csv
['/content/drive/MyDrive/colab_projects/empathic-agent-clean/experiments/experiment_2_empathy_agent/output/llama2_lora_4bit_small_reference_based_score.csv', '/content/drive/MyDrive/colab_projects/empathic-agent-clean/experiments/experiment_2_empathy_agent/output/llama2_lora_4bit_small_inference.csv']


# Interpreting the scores (per the paper arXiv:2503.16545)
Overlap: shared actions ratio (higher = more actions in common).
LCS: longest common action subsequence (order-sensitive; higher is better).
TF‑IDF: weighted action similarity (higher = closer to ground-truth actions).

# 2) Generate Scenario Understanding (L1) with your LoRA (free-text; EmpathyAgent scores with BERTScore)

In [ ]:
# L1 generation with LoRA (free text)
import os, json, csv, torch
from pathlib import Path
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

PROJECT_ROOT = "/content/drive/MyDrive/colab_projects/empathic-agent-clean"
BASE_MODEL   = "meta-llama/Llama-2-7b-chat-hf"
ADAPTER_DIR  = f"{PROJECT_ROOT}/experiments/experiment_2_empathy_agent/output/llama2_lora_4bit"
TEST_JSON    = f"{PROJECT_ROOT}/experiments/experiment_2_empathy_agent/code/EmpathyAgent/dataset/testset_100.json"
PROMPT_L1    = f"{PROJECT_ROOT}/experiments/experiment_2_empathy_agent/code/EmpathyAgent/baseline/prompt/prompt_video_l1.txt"
OUT_CSV      = f"{PROJECT_ROOT}/experiments/experiment_2_empathy_agent/output/llama2_lora_4bit_small_L1_inference.csv"

def load_json(p):
    for enc in ("utf-8","utf-8-sig","latin-1"):
        try:
            with open(p,"r",encoding=enc) as f: return json.load(f)
        except UnicodeDecodeError: pass
    return json.loads(open(p,"rb").read().decode("utf-8",errors="ignore"))

data = load_json(TEST_JSON)
prompt_tmpl = Path(PROMPT_L1).read_text(encoding="utf-8").strip() if Path(PROMPT_L1).exists() else (
"""You are an empathetic assistant. Given the character info and dialogue, write a concise scenario understanding (1–3 sentences) that:
- identifies the person’s emotional state,
- explains likely cause/context,
- notes key objects and relations.
Be specific and grounded, avoid actions, avoid lists.

Character:
{character_info}

Dialogue:
{dialogue}

Your scenario understanding:"""
)

# 4-bit base + adapter (device_map auto offload)
bnb = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_use_double_quant=True)
tok = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True); tok.pad_token = tok.eos_token
base = AutoModelForCausalLM.from_pretrained(BASE_MODEL, quantization_config=bnb, torch_dtype=torch.float16, device_map="auto", low_cpu_mem_usage=True)
model = PeftModel.from_pretrained(base, ADAPTER_DIR); model.eval()

def build_l1_prompt(x):
    character_info = x.get("character_info","") or x.get("character","")
    dialogue = x.get("dialogue","")
    return prompt_tmpl.format(character_info=character_info, dialogue=dialogue)

os.makedirs(Path(OUT_CSV).parent, exist_ok=True)
with torch.inference_mode():
    with open(OUT_CSV,"w",newline="",encoding="utf-8") as f:
        w = csv.writer(f); w.writerow(["data_idx","response"])
        for i, x in enumerate(data):
            ip  = build_l1_prompt(x)
            inp = tok(ip, return_tensors="pt", truncation=True, max_length=1024).to(model.device)
            out = model.generate(**inp, max_new_tokens=120, temperature=0.2, top_p=0.9, do_sample=True)
            gen_ids = out[0][inp["input_ids"].shape[1]:]  # slice off the prompt
            resp = tok.decode(gen_ids, skip_special_tokens=True).strip()
            w.writerow([i, resp])
            if (i+1) % 10 == 0: print(f"generated {i+1}/{len(data)}")
print("Wrote:", OUT_CSV)

# Score L1 with BERTScore (reference-based)

In [ ]:
# L1 BERTScore (reference-based)
import os, csv, json
from pathlib import Path
from bert_score import score

PROJECT_ROOT = "/content/drive/MyDrive/colab_projects/empathic-agent-clean"
TEST_JSON    = f"{PROJECT_ROOT}/experiments/experiment_2_empathy_agent/code/EmpathyAgent/dataset/testset_100.json"
IN_CSV       = f"{PROJECT_ROOT}/experiments/experiment_2_empathy_agent/output/llama2_lora_4bit_small_L1_inference.csv"
OUT_SCORE    = f"{PROJECT_ROOT}/experiments/experiment_2_empathy_agent/output/llama2_lora_4bit_small_L1_reference_based_score.csv"

def load_json(p):
    for enc in ("utf-8","utf-8-sig","latin-1"):
        try:
            with open(p,"r",encoding=enc) as f: return json.load(f)
        except UnicodeDecodeError: pass
    return json.loads(open(p,"rb").read().decode("utf-8",errors="ignore"))

data = load_json(TEST_JSON)
refs = [ (x.get("scenario_understanding") or x.get("reference") or "").strip() for x in data ]  # adapt to dataset field
gens = [""] * len(data)

with open(IN_CSV, encoding="utf-8") as f:
    for r in csv.DictReader(f):
        i = (r.get("data_idx") or "").strip()
        t = (r.get("response") or "").strip()
        if i.isdigit():
            gens[int(i)] = t

# keep aligned pairs only
pairs = [(g, r) for g, r in zip(gens, refs)]
gens2, refs2 = zip(*pairs)

P, R, F1 = score(list(gens2), list(refs2), lang="en", model_type="roberta-large", verbose=False)
val = float(F1.mean().item())
print("BERTScore (F1):", val)

with open(OUT_SCORE, "w", newline="", encoding="utf-8") as f:
    w = csv.writer(f); w.writerow(["Metric","Score"]); w.writerow(["BERTScore", val])
print("Wrote:", OUT_SCORE)